<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

In [2]:
#Packages
#--Web scraping packages
from bs4 import BeautifulSoup
import requests
#Pandas/numpy for data manipulation
import pandas as pd
import numpy as np

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [3]:
#load URLs we want to scrape into an array
BASE_URL = [
'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
]

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
postcode_members = []
#Loop through our URLs we loaded above
for b in BASE_URL:
    html = requests.get(b).text
    soup = BeautifulSoup(html, "html.parser")
    #identify table we want to scrape
    #wiki_table = soup.find('table', {"class" : "dataTable"})
    wiki_table = soup.find('tbody')
    #try clause to skip any companies with missing/empty board member tables
    try:
        #loop through table, grab each of the 4 columns shown (try one of the links yourself to see the layout)
        for row in wiki_table.find_all('tr'):
            cols = row.find_all('td')
            #print(len(cols))
            if len(cols) == 3:
                postcode_members.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))                
    except: pass  

In [5]:
#convert output to new array, check length
postcode_array = np.asarray(postcode_members)
len(postcode_array)

287

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [6]:
#convert new array to dataframe
df = pd.DataFrame(postcode_array)
df.columns = ['PostalCode', 'Borough','Neighborhood']

In [7]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
df = df.drop(df[df.Borough == "Not assigned"].index)
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


More than one neighborhood can exist in one postal code area. These rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    

In [9]:
df['Neighborhood'] = df[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
df[['PostalCode','Borough','Neighborhood']].drop_duplicates()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
7,M7A,Queen's Park,Not assigned
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,"Rouge,Malvern"
13,M3B,North York,Don Mills North
14,M4B,East York,"Woodbine Gardens,Parkview Hill"
16,M5B,Downtown Toronto,"Ryerson,Garden District"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [10]:
#df[ (df.Neighborhood=='Not assigned') ]['Neighborhood'] = df.Borough
df['Neighborhood'] = np.where((df.Neighborhood == 'Not assigned'),df.Borough,df.Neighborhood)
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
7,M7A,Queen's Park,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,"Rouge,Malvern"
11,M1B,Scarborough,"Rouge,Malvern"
13,M3B,North York,Don Mills North


In [11]:
#Print the number of rows in the dataframe
df.shape

(210, 3)

In [12]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_data_1.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [19]:
df_merge = pd.merge(df, df_data_1, on=['PostalCode'], how='left')

In [20]:
df_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
